## Imports

In [36]:
!pip install -q z3
!pip install -q z3-solver

In [37]:
from tqdm import tqdm
from z3 import Int, Solver

In [38]:
day = "24"

In [39]:
with open(f"input_j{day}.txt") as f:
# with open("test.txt") as f:
    stones = f.readlines()
    stones = list(map(lambda x: x.replace("\n", ""), stones))

n = len(stones)
n

300

In [40]:
for i in range(n):
    position = stones[i].split(" @ ")[0].split(", ")
    vitesse = stones[i].split(" @ ")[1].split(", ")
    stones[i] = {
        "x": int(position[0]),
        "y": int(position[1]),
        "z": int(position[2]),
        "vx": int(vitesse[0]),
        "vy": int(vitesse[1]),
        "vz": int(vitesse[2])
    }

In [41]:
stones[0]

{'x': 309254625334097,
 'y': 251732589486275,
 'z': 442061964691135,
 'vx': -42,
 'vy': -22,
 'vz': -45}

## Part 1

In [42]:
def intersection_2d(stone_1, stone_2):

    x1, y1, vx1, vy1 = stone_1['x'], stone_1['y'], stone_1['vx'], stone_1['vy']
    x2, y2, vx2, vy2 = stone_2['x'], stone_2['y'], stone_2['vx'], stone_2['vy']

    if (x1 == x2) and (y1 == y2):
        return (x1, y1)  # Points superposés

    if vx1 * vy2 == vy1 * vx2:

        if ((x2 - x1) * vy1 == (y2 - y1) * vx1) and (vx1 * vx2 <= 0) and (vy1 * vy2 <= 0):
            # Trajectoires parallèles, mais colinéaires et se dirigeant l'une vers l'autre
            t_intersection = (x2 - x1) / (vx1 - vx2)
            if t_intersection < 0:
                return (float("inf"), float("inf"))
            x_intersection = x1 + vx1 * t_intersection
            y_intersection = y1 + vy1 * t_intersection
            return (x_intersection, y_intersection)

        return (float("inf"), float("inf"))  # Trajectoires parallèles et pas d'intersection

    t_intersection_1 = ((x1 - x2) * vy2 - (y1 - y2) * vx2) / (vy1 * vx2 - vx1 * vy2)
    t_intersection_2 = ((x1 - x2) * vy1 - (y1 - y2) * vx1) / (vx2 * vy1 - vy2 * vx1)
    if t_intersection_1 < 0 or t_intersection_2 < 0:
        return (float("inf"), float("inf"))
    # print(t_intersection_1, t_intersection_2)
    x_intersection = x1 + vx1 * t_intersection_1
    y_intersection = y1 + vy1 * t_intersection_1

    return (x_intersection, y_intersection)  # Trajectoires sécantes


In [43]:
def are_stones_colliding_within_area(stone_1, stone_2,
                                     x_min=200000000000000,
                                     x_max=400000000000000,
                                     y_min=200000000000000,
                                     y_max=400000000000000):
    x_intersection, y_intersection = intersection_2d(stone_1, stone_2)
    return (
        (
            (x_intersection >= x_min) and (x_intersection <= x_max)
        ) and (
            (y_intersection >= y_min) and (y_intersection <= y_max)
        )
    )


In [44]:
compteur = 0
for i in tqdm(range(n)):
    for j in range(i+1, n):
        # print(i, j, intersection_2d(stones[i], stones[j]))
        if are_stones_colliding_within_area(stones[i], stones[j]):
            compteur += 1


100%|██████████| 300/300 [00:00<00:00, 3480.48it/s]


In [45]:
compteur

31921

### Part 2

In [46]:
stone_ex = {'x': 24,
 'y': 13,
 'z': 10,
 'vx': -3,
 'vy': 1,
 'vz': 2}

In [47]:
def intersection_3d(stone_1, stone_2):

    x1, y1, z1, vx1, vy1, vz1 = stone_1['x'], stone_1['y'], stone_1['z'], stone_1['vx'], stone_1['vy'], stone_1['vz']
    x2, y2, z2, vx2, vy2, vz2 = stone_2['x'], stone_2['y'], stone_2['z'], stone_2['vx'], stone_2['vy'], stone_2['vz']

    if (x1 == x2) and (y1 == y2) and (z1 == z2):
        return (x1, y1, z1)  # Points superposés

    det_xy = vy1 * vx2 - vx1 * vy2
    det_xz = vz1 * vx2 - vx1 * vz2
    det_yz = vy1 * vz2 - vz1 * vy2

    if det_xy == 0 and det_xz == 0 and det_yz == 0:
        if (
            (x2 - x1) * vy1 == (y2 - y1) * vx1
        ) and (
            (x2 - x1) * vz1 == (z2 - z1) * vx1
        ) and (
            (y2 - y1) * vz1 == (z2 - z1) * vy1
        ) and (
            (vx1 * vx2 <= 0) and (vy1 * vy2 <= 0) and (vz1 * vz2 <= 0)
        ):
            # Trajectoires parallèles, colinéaires et se dirigeant l'une vers l'autre
            t_intersection = (x2 - x1) / (vx1 - vx2)
            if t_intersection < 0:
                return (float("inf"), float("inf"), float("inf"))
            x_intersection = x1 + vx1 * t_intersection
            y_intersection = y1 + vy1 * t_intersection
            z_intersection = z1 + vz1 * t_intersection
            return (x_intersection, y_intersection, z_intersection)

        return (float("inf"), float("inf"), float("inf"))  # Trajectoires parallèles et pas d'intersection

    # Calcul des temps d'intersection
    t_intersection_1 = ((x1 - x2) * vy2 - (y1 - y2) * vx2) / det_xy if det_xy != 0 else 0
    t_intersection_2 = ((x1 - x2) * vz2 - (z1 - z2) * vx2) / det_xz if det_xz != 0 else 0
    t_intersection_3 = ((z1 - z2) * vy2 - (y1 - y2) * vz2) / det_yz if det_yz != 0 else 0
    # print(t_intersection_1, t_intersection_2, t_intersection_3)
    # Vérifier si les temps d'intersection sont positifs
    if t_intersection_1 < 0 or t_intersection_2 < 0 or t_intersection_3 < 0:
        return (float("inf"), float("inf"), float("inf"))

    x_intersection = int(x1 + vx1 * t_intersection_1)
    y_intersection = int(y1 + vy1 * t_intersection_1)
    z_intersection = int(z1 + vz1 * t_intersection_1)

    return (x_intersection, y_intersection, z_intersection)  # Trajectoires sécantes


In [48]:
# for stone in stones:
#     print(intersection_3d(stone, stone_ex))

Brut force searching of solutions did not work (wayyyyy too long).

I had to go on Reddit to see that people were using a package called Z3 to do the numerical solving of the system, that I thought was too complicated by hand. I was not able to find a more proper solution, so I will juste use the documentation of the package and go through with it: https://ericpony.github.io/z3py-tutorial/guide-examples.htm.

In [49]:
x, y, z, vx, vy, vz = Int('x'), Int('y'), Int('z'), Int('vx'), Int('vy'), Int('vz')
times = [Int(f"t_{i}") for i in range(n)]

solver = Solver()
for i in range(n):
  solver.add(x + times[i]*vx - stones[i]['x'] - times[i]*stones[i]['vx'] == 0)
  solver.add(y + times[i]*vy - stones[i]['y'] - times[i]*stones[i]['vy'] == 0)
  solver.add(z + times[i]*vz - stones[i]['z'] - times[i]*stones[i]['vz'] == 0)

solver.check()

sat

In [50]:
modele = solver.model()
modele.eval(x+y+z)

761691907059631